In [87]:
%cd /home/parthgandhi/TradeBot

/home/parthgandhi/TradeBot


In [88]:
import polars as pl
import polars.selectors as cs
from src.config.storage_layout import StorageLayout
from src.config.market import Market
from src.config.exchange import Exchange
from src.config.data_source import DataSource
from src.config.brokers.nse import NSEConfig
from src.data_source.chartsmaze.helper import industry_to_sector
from src.data_source.chartsmaze.sectors import sectors as cmze_sectors

In [99]:
END_DATE = "2025-12-26"
RS_RATING = 70

In [91]:
#  rclone sync /home/parthgandhi/TradeBot/storage/data/IND/ChartsMaze gdrive:Backup/SwingTrade/ChartsMaze/RS --progress

# Industry Analysis

In [ ]:
analysis_path = StorageLayout.analysis_dir(run_date=END_DATE, market=Market.INDIA_EQUITIES, exchange=Exchange.NSE)

res = pl.scan_csv(analysis_path / "overall_filter_result.csv").filter(
    pl.col("rs_rating") >= RS_RATING
)

In [ ]:
industry_analysis = (
    res.lazy()
    .group_by("sector_cmaze","basic_industry_cmaze")
    .agg(
        [pl.col("symbol").count().alias("symbols_count")]
        + [
            (
                (pl.col(f"{i}_mo_rtr_pct") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias(f"{i}_mo_avg_weight_rtr_pct")
            for i in [1, 3]
        ]
        + [
            (
                (pl.col("rs_rating") * pl.col("market_cap_cr_cmaze")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias(f"rs_rating_avg_weight")
        ]
    )
    .fill_nan(None)
    .with_columns(
        pl.mean_horizontal(cs.exclude("sector_cmaze", "basic_industry_cmaze", "symbols_count"))
        .round(2)
        .alias("industry_score"),
        (pl.col("1_mo_avg_weight_rtr_pct") * 100 / pl.col("3_mo_avg_weight_rtr_pct"))
        .round()
        .alias("1_by_3_rtr"),
    ).sort("symbols_count", descending=True)
    .collect()
)

industry_analysis.write_csv(analysis_path / "industry_analysis.csv")

# RS >= 70 & Possible Upcoming Centers

In [16]:
industry_analysis.filter(
    (pl.col("rs_rating_avg_weight") >= 70)
    & (pl.col("1_mo_avg_weight_rtr_pct") >= pl.col("3_mo_avg_weight_rtr_pct"))
).sort("industry_score", descending=True)

basic_industry,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,u32,f64,f64,f64,f64,f64
"""Media & Entertainment""",1,43.0,33.5,99.0,58.5,128.0
"""Packaging""",1,17.2,6.3,93.0,38.83,273.0
"""Railways""",1,19.9,14.8,73.0,35.9,134.0
"""Oil & Gas Drilling""",1,15.6,8.8,76.0,33.47,177.0


In [18]:
industry_analysis.filter((pl.col("rs_rating_avg_weight") >= 70)).sort(
    "industry_score", descending=True
).filter(pl.col("symbols_count") > 1)

basic_industry,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,u32,f64,f64,f64,f64,f64
"""Mining/Minerals""",8,25.44,35.95,92.61,51.33,71.0
"""NBFC""",10,7.77,40.32,95.3,47.8,19.0
"""Tyres & Rubber Products""",3,8.16,35.67,93.29,45.71,23.0
"""Construction Products Miscalla…",3,9.42,32.32,88.84,43.53,29.0
"""Diversified Commercial Service…",3,10.17,30.77,85.78,42.24,33.0
…,…,…,…,…,…,…
"""Investment Banking & Broking""",5,3.82,9.87,77.13,30.27,39.0
"""Pharmaceuticals""",9,1.56,7.78,80.78,30.04,20.0
"""Tea & Coffee""",2,-7.4,5.99,88.77,29.12,-124.0
